In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import time
import fuzzymatcher
from ftfy import fix_text 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from scipy.sparse import rand
import sparse_dot_topn.sparse_dot_topn as ct

In [3]:
checkbook = pd.read_csv('checkbook_nyc_groupby_capital_project_fy.csv')
cpdb = pd.read_csv('../data/cpdb_projects.csv')

In [12]:
cpdb.head(5)

,ccpversion,magency,projectid,maprojid,description,ccnonexempt,ccexempt,citycost,nccstate,nccfederal,nccother,noncitycost,totalcost,magencyacro,magencyname
0,fisa_2023,35,L103RENO,035L103RENO,RESEARCH LIBRARIES - Renovations ...,0,0,0,0,0,0,0,0,NYRL,New York Research Libraries
1,fisa_2023,35,L19TECHUP,035L19TECHUP,Technology Upgrades - Research Libraries ...,0,0,0,0,0,0,0,0,NYRL,New York Research Libraries
2,fisa_2023,35,L20RTECH,035L20RTECH,Research Libraries - Technology Upgrades ...,20000,0,20000,0,0,0,0,20000,NYRL,New York Research Libraries
3,fisa_2023,35,L21EXENNS,035L21EXENNS,NYPL Research Libraries - FY 2022 Executive Pl...,0,0,0,0,0,0,0,0,NYRL,New York Research Libraries
4,fisa_2023,35,L21FREEZE,035L21FREEZE,NYPL Research Libraries - Blast Freezer ...,400000,0,400000,0,0,0,0,400000,NYRL,New York Research Libraries


In [8]:
checkbook_2023 = checkbook[checkbook['Fiscal year']==2023]
checkbook_2023.shape

(6442, 21)

In [13]:
start = time.time()
matched = fuzzymatcher.fuzzy_left_join(checkbook_2023, cpdb, left_on='FMS ID', right_on='projectid')
matched = matched[matched['best_match_score'] > 0.3]
end = time.time()

print("time elapsed: {}".format(end-start))

time elapsed: 9.047394037246704


In [16]:
matched.shape[0]

matched.head(10)

,best_match_score,__id_left,__id_right,Unnamed: 0,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,...,ccnonexempt,ccexempt,citycost,nccstate,nccfederal,nccother,noncitycost,totalcost,magencyacro,magencyname
173,0.410728,167_left,5632_right,1750,SCHOOL CONSTRUCTION AUTHORITY,NaN,SEWA (GREEN INFRASTRUCTURE PLAYGROUNDS),040GI-DOESCA020,4401509.88,PON104420232307182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DEP,Department of Environmental Protection
1635,0.373824,1399_left,1104_right,15194,Fire Department,NaN,ESAD (FDNY: EAST SIDE ACCESS RADIO COMMUNICATI),057F1-ESADAS001,398668.14,CT105720181408073,...,3302000.0,0.0,3302000.0,0.0,0.0,0.0,0.0,3302000.0,FDNY,Fire Department
1641,0.349100,1405_left,1135_right,15299,Fire Department,NaN,"RM21 (FDNY, BUREAU OF FLEET SVCS, 48-58 34TH S)",057F109-RM21001,1348406.76,DO185720212155366,...,23987000.0,0.0,23987000.0,0.0,0.0,0.0,0.0,23987000.0,FDNY,Fire Department
1873,0.302794,1612_left,1669_right,18197,Department of Correction,NaN,HVAC (DOC: UPGRADES TO & INSTALLATION OF VENTI),072C75NICA/C014,232910.39,CT107220111423673,...,464000.0,0.0,464000.0,0.0,0.0,0.0,0.0,464000.0,DOC,Department of Correction
2254,0.307931,1732_left,2075_right,20126,Police Department,NaN,MP22 (NYPD ARGUS CAMERA SYSTEM EXPANSION MULT.),111PO-M185B 001,39880.00,NaN,...,10000.0,0.0,10000.0,0.0,0.0,0.0,0.0,10000.0,UK,Unknown
2256,0.307931,1734_left,2077_right,20128,Police Department,NaN,MP22 (NYPD ARGUS CAMERA SYSTEM EXPANSION MULT.),111PO-M185D 001,23698.83,NaN,...,10000.0,0.0,10000.0,0.0,0.0,0.0,0.0,10000.0,UK,Unknown
2258,0.307931,1736_left,2083_right,20145,Police Department,NaN,ARSI (NYPD: PURCHASE OF ARGUS CCTV SECURITY CA),111PO-R185F 002,53233.75,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UK,Unknown
2329,0.307931,1754_left,2212_right,20591,Police Department,NaN,VCM5 (NYPD LABORATORY FORENSIC INVESTIGATION S),111PO187-VCM003,348637.20,CT185720228804265,...,1025000.0,0.0,1025000.0,0.0,0.0,0.0,0.0,1025000.0,UK,Unknown
2380,0.303835,1795_left,2744_right,21424,Department of Cultural Affairs,NaN,"ADHA (HARLEM STAGE, INC., AARON DAVIS HALL GAT)",126PV829-ADH001,37248.55,CT112620238804725,...,114000.0,0.0,114000.0,0.0,0.0,0.0,0.0,114000.0,DCLA,Department of Cultural Affairs
2381,0.303835,1796_left,2744_right,21425,Department of Cultural Affairs,NaN,"ADHB (HARLEM STAGE, INC., AARON DAVIS HALL GAT)",126PV829-ADH002,12553.59,CT112620238804725,...,114000.0,0.0,114000.0,0.0,0.0,0.0,0.0,114000.0,DCLA,Department of Cultural Affairs
